In [ ]:
#/usr/local/lib/python3.7/dist-packages/torch/nn/modules/unsampling.py
# def forward(self, input: Tensor) -> Tensor:
#         return F.interpolate(input, self.size, self.scale_factor, self.mode, self.align_corners)
# !pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.4 MB/s 


In [ ]:
# from google.colab import drive
# #drive.mount('/content/drive',force_remount=True)
# drive.mount('/content/drive')
# %cd /MyDrive/file

Mounted at /content/drive
[Errno 2] No such file or directory: '/MyDrive/file'
/content


In [ ]:
nor=[
    [1612.2641794179053,694.6404158569574],
    [1379.8899556061613,734.589213934987],
    [1344.4295633683826,731.6118897277566],
    [1195.157229000143,860.603745394514],
    [1439.168369746529,771.3569863637912],
    [2344.2498120705645,921.6300590130161],
    [2796.473722876989,1088.0256714514674],
    [2578.4108992777597,1029.246558060433],
    [3023.817505678254,1205.1064480965915],
    [476.7287418382585,331.6878880293502],
    [59.24111403905757,130.40242222578226],
    [1989.1945548720423,993.7071664926801],
    [1152.4886461779677, 768.8907975412457],
    [-0.2938129263276281,0.21578320121968173],
    [-0.36928344017880277,0.19538602918264955],
    [-6393.00646782349,0.19538602918264955],
    [-2398.566478742078,0.19538602918264955]
]

In [ ]:
from osgeo import gdal
import torch
from torchvision import transforms
import torchvision.transforms.functional as F
import torchvision.models as models
import torch.nn as nn

import numpy as np
import random
from PIL import Image
from time import time
import csv
import os
import numpy as np
import rasterio
import argparse

from tqdm.notebook import tqdm
from IPython.display import clear_output

import os
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
LR = 5e-4
EPOCHS = 1
EPOCHS_PER_UPDATE = 1

CHANNEL=5
outputnumber=3

path='./dataset/v1.1/data/flood_events/HandLabeled/'  #修改
model_save_path='./checkpoints/'  #修改
# train
S2_path = path+'S2Hand'
Labels_path = path+'LabelHand'
# test
test_path = './dataset/test/post/'

In [ ]:
class VGGBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, middle_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.conv2 = nn.Conv2d(middle_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        return out

class NestedUNet(nn.Module):
    def __init__(self, num_classes, input_channels=3, deep_supervision=False, **kwargs):
        super().__init__()

        nb_filter = [32, 64, 128, 256, 512,1024]

        self.deep_supervision = deep_supervision

        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])
        

        self.conv0_1 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])
        

        self.conv0_2 = VGGBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = VGGBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = VGGBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], nb_filter[2])
        

        self.conv0_3 = VGGBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = VGGBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], nb_filter[1])
        

        self.conv0_4 = VGGBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], nb_filter[0])
        

       



        if self.deep_supervision:
            self.final1 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final2 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final3 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final4 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            
        else:
            self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)


    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            
            if outputnumber==0:
              return output1
            elif outputnumber==1:
              return output2
            elif outputnumber==2:
              return output3
            elif outputnumber==3:
              return output4
            

         
        else:
            output = self.final(x0_4)
            return output

In [ ]:
def processTestIm(data):
  (sen2, y) = data
  s2, label = sen2.copy(), y.copy()

  norm = transforms.Normalize([nor[6][0],
         nor[7][0],
         nor[8][0],
         nor[11][0],
         nor[12][0]],[nor[6][1],
         nor[7][1],
         nor[8][1],
         nor[11][1],
         nor[12][1]])


  # convert to PIL for easier transforms
  

  band1 = Image.fromarray(s2[0]).resize((512,512))

  band2 = Image.fromarray(s2[1]).resize((512,512))
  # print(band2.mean())
  band3 = Image.fromarray(s2[2]).resize((512,512))
  band4 = Image.fromarray(s2[3]).resize((512,512))
  band5 = Image.fromarray(s2[4]).resize((512,512))
  band6 = Image.fromarray(s2[5]).resize((512,512))
  band7 = Image.fromarray(s2[6]).resize((512,512))
  band8 = Image.fromarray(s2[7]).resize((512,512))
  # print(band8.mean())
  band8A = Image.fromarray(s2[8]).resize((512,512))
  band9 = Image.fromarray(s2[9]).resize((512,512))
  # print(band10.mean())
  band10 = Image.fromarray(s2[10]).resize((512,512))
  band11 = Image.fromarray(s2[11]).resize((512,512))
  # print(band12.mean())
  band12 = Image.fromarray(s2[12]).resize((512,512))

  
  label = Image.fromarray(label.squeeze()).resize((512,512))


 


  band1s = [F.crop(band1, 0, 0, 256, 256), F.crop(band1, 0, 256, 256, 256),
              F.crop(band1, 256, 0, 256, 256), F.crop(band1, 256, 256, 256, 256)]
  band2s = [F.crop(band2, 0, 0, 256, 256), F.crop(band2, 0, 256, 256, 256),
              F.crop(band2, 256, 0, 256, 256), F.crop(band2, 256, 256, 256, 256)]
  band3s = [F.crop(band3, 0, 0, 256, 256), F.crop(band3, 0, 256, 256, 256),
              F.crop(band3, 256, 0, 256, 256), F.crop(band3, 256, 256, 256, 256)]
  band4s = [F.crop(band4, 0, 0, 256, 256), F.crop(band4, 0, 256, 256, 256),
              F.crop(band4, 256, 0, 256, 256), F.crop(band4, 256, 256, 256, 256)]
  band5s = [F.crop(band5, 0, 0, 256, 256), F.crop(band5, 0, 256, 256, 256),
              F.crop(band5, 256, 0, 256, 256), F.crop(band5, 256, 256, 256, 256)]
  band6s = [F.crop(band6, 0, 0, 256, 256), F.crop(band6, 0, 256, 256, 256),
              F.crop(band6, 256, 0, 256, 256), F.crop(band6, 256, 256, 256, 256)]
  band7s = [F.crop(band7, 0, 0, 256, 256), F.crop(band7, 0, 256, 256, 256),
              F.crop(band7, 256, 0, 256, 256), F.crop(band7, 256, 256, 256, 256)]
  band8s = [F.crop(band8, 0, 0, 256, 256), F.crop(band8, 0, 256, 256, 256),
              F.crop(band8, 256, 0, 256, 256), F.crop(band8, 256, 256, 256, 256)]
  band9s = [F.crop(band9, 0, 0, 256, 256), F.crop(band9, 0, 256, 256, 256),
              F.crop(band9, 256, 0, 256, 256), F.crop(band9, 256, 256, 256, 256)]
  band10s = [F.crop(band10, 0, 0, 256, 256), F.crop(band10, 0, 256, 256, 256),
              F.crop(band10, 256, 0, 256, 256), F.crop(band10, 256, 256, 256, 256)]
  band11s = [F.crop(band11, 0, 0, 256, 256), F.crop(band11, 0, 256, 256, 256),
              F.crop(band11, 256, 0, 256, 256), F.crop(band11, 256, 256, 256, 256)]
  band12s = [F.crop(band12, 0, 0, 256, 256), F.crop(band12, 0, 256, 256, 256),
              F.crop(band12, 256, 0, 256, 256), F.crop(band12, 256, 256, 256, 256)]
  band8As = [F.crop(band8A, 0, 0, 256, 256), F.crop(band8A, 0, 256, 256, 256),
              F.crop(band8A, 256, 0, 256, 256), F.crop(band8A, 256, 256, 256, 256)]
 
  labels = [F.crop(label, 0, 0, 256, 256), F.crop(label, 0, 256, 256, 256),
            F.crop(label, 256, 0, 256, 256), F.crop(label, 256, 256, 256, 256)]



  
  ims = [torch.stack((transforms.ToTensor()(x1).squeeze().float(),
                    transforms.ToTensor()(x2).squeeze().float(),
                    transforms.ToTensor()(x3).squeeze().float(),
                    transforms.ToTensor()(x4).squeeze().float(),
                    transforms.ToTensor()(x5).squeeze().float()))
                    for (x1,x2,x3,x4,x5,) in zip(band7s,band8s,band8As,band11s,band12s)]

  
  
  ims = [norm(im) for im in ims]
  ims = torch.stack(ims)
  
  labels = [(transforms.ToTensor()(label).squeeze()) for label in labels]
  labels = torch.stack(labels)
  # print(labels)
  
  if torch.sum(labels.gt(.003) * labels.lt(.004)):
    labels *= 255
  labels=labels/1.0
  labels = labels.round()
  
  return ims, labels


In [ ]:
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1,8]).float().cuda(), ignore_index=255) 

def computeIOU(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  intersection = torch.sum(output * target)
  union = torch.sum(target) + torch.sum(output) - intersection
  iou = (intersection + .0000001) / (union + .0000001)
  
  if iou != iou:
    print("failed, replacing with 0")
    iou = torch.tensor(0).float()
  
  return iou
  
def computeAccuracy(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output.eq(target))
  
  return correct.float() / len(target)

In [ ]:
def getArrFlood(fname):
    return rasterio.open(fname).read()

def download_flood_test_data_from_list(l):
  
  i = 0
  tot_nan = 0
  tot_good = 0
  flood_data = []
  for (s2_fname,mask_fname) in l:

    if not os.path.exists(s2_fname):
      print("(((((")
      continue

    # arr_vv = np.nan_to_num(getArrFlood(vv_fname))
    # arr_vh=np.nan_to_num(getArrFlood(vh_fname))
    # arr_s1 = np.zeros((2,512,512), dtype=float)
    # arr_s1[0,:,:]=arr_vv
    # arr_s1[1,:,:]=arr_vh
    #print(arr_s1.shape)

    arr_s2 = np.nan_to_num(getArrFlood(s2_fname))[1:,:,:]
    
    arr_y = getArrFlood(mask_fname)
    arr_y[arr_y == -1] = 255

    # arr_s1 = np.clip(arr_s1, -50, 1)
    # arr_s1 = (arr_s1 + 50) / 51

    if i % 100 == 0:
      # print(vv_fname,vh_fname, mask_fname)
      print(s2_fname, mask_fname)
    i += 1
    flood_data.append((arr_s2,arr_y))

  return flood_data
 

def load_flood_test_data(path):
  input_root_s2 =path+'S2/'
  label_root=path+'Labels/'
  fname = path+"flood_test_data.csv"
  training_files = []
  with open(fname) as f:
    for line in csv.reader(f):
      training_files.append(tuple((input_root_s2+line[2], label_root+line[3])))
      
  print(training_files)

  return download_flood_test_data_from_list(training_files)

In [ ]:
test_data = load_flood_test_data(test_path)
print(len(test_data))

In [ ]:
def test_loop(test_data, net):
    net = net.eval()
    net = net.cuda()
    count = 0
    iou = 0
    loss = 0
    accuracy = 0
    
    with torch.no_grad():
        print(len(test_data))
        for i in range(0,len(test_data)):
            (images,labels)=processTestIm(test_data[i])
        
            net = net.cuda()
            
            print(i)
            outputs = net(images.cuda())
            valid_loss = criterion(outputs, labels.long().cuda())
            valid_iou = computeIOU(outputs, labels.cuda())
            iou += valid_iou
            accuracy += computeAccuracy(outputs, labels.cuda())
            count += 1

            output = torch.argmax(outputs, dim=1).flatten() 

            output=output.cpu().numpy()

            output=np.array(output)
            output=output.reshape((4,256,256))
            image=images.reshape((4,5,256,256))

            label = labels.cuda().flatten()
            #print(label.shape)
            label=label.cpu().numpy()
            #print(label.shape)
            label=np.array(label)
            label=label.reshape((4,256,256))

            for i in range(0,4):

                # x1 = np.roll(output[i].cpu(),-1,0)
                #print(output[i].shape)


                _, ax = plt.subplots(1, 5,)
                ax[0].imshow(output[i])
                ax[0].set_title('pred')

                ax[1].imshow(label[i])
                ax[1].set_title('label')
                
                ax[2].imshow(image[i][2])
                ax[2].set_title('s2')

                ax[3].imshow(image[i][3])
                ax[3].set_title('s2')

                ax[4].imshow(image[i][4])
                ax[4].set_title('s2')



                plt.show()

    iou = iou / count
    print("Test IOU:", iou)
    print("Test Accuracy:", accuracy / count)

In [ ]:
model_name = 'Perus_part_s2_3_4_0.5840604901313782.cp'

model_path = model_save_path + model_name
model=torch.load(model_path)
test_loop(test_data, model)